In [10]:
import argparse
import pandas as pd
import tensorflow as tf
from src.Predict import XGBoost_Runner
from src.Utils.Dictionaries import team_index_current
from src.Utils.tools import (get_json_data, to_data_frame, get_todays_games_json,create_todays_games)

In [4]:
todays_games_url = 'https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/scores/00_todays_scores.json'
data_url = 'https://stats.nba.com/stats/leaguedashteamstats?' \
           'Conference=&DateFrom=&DateTo=&Division=&GameScope=&' \
           'GameSegment=&LastNGames=0&LeagueID=00&Location=&' \
           'MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&' \
           'PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&' \
           'PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&' \
           'Season=2021-22&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&' \
           'StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

In [5]:
def createTodaysGames(games, df):
    match_data = []
    todays_games_uo = []
    home_team_odds = []
    away_team_odds = []

    for game in games:
        home_team = game[0]
        away_team = game[1]
        todays_games_uo.append(input(home_team + ' vs ' + away_team + ': '))

        home_team_odds.append(input(home_team + ' odds: '))
        away_team_odds.append(input(away_team + ' odds: '))

        home_team_series = df.iloc[team_index_current.get(home_team)]
        away_team_series = df.iloc[team_index_current.get(away_team)]
        stats = home_team_series.append(away_team_series)
        match_data.append(stats)

    games_data_frame = pd.concat(match_data, ignore_index=True, axis=1)
    games_data_frame = games_data_frame.T

    frame_ml = games_data_frame.drop(columns=['TEAM_ID', 'CFID', 'CFPARAMS', 'TEAM_NAME'])
    data = frame_ml.values
    data = data.astype(float)

    return data, todays_games_uo, frame_ml, home_team_odds, away_team_odds

In [6]:
data = get_todays_games_json(todays_games_url)

In [11]:
games = create_todays_games(data)

In [13]:
data = get_json_data(data_url)

In [15]:
df = to_data_frame(data)

In [18]:
df

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,1610612737,Atlanta Hawks,40,17,23,0.425,48.0,40.6,88.4,0.459,...,1,29,25,13,3,13,10,22,10,Atlanta Hawks
1,1610612738,Boston Celtics,42,21,21,0.500,49.0,39.0,87.8,0.445,...,16,15,2,14,8,12,19,10,10,Boston Celtics
2,1610612751,Brooklyn Nets,40,26,14,0.650,48.3,41.3,87.8,0.471,...,18,21,7,20,20,8,7,9,10,Brooklyn Nets
3,1610612766,Charlotte Hornets,42,23,19,0.548,48.5,42.9,92.5,0.464,...,6,4,8,18,16,18,2,17,10,Charlotte Hornets
4,1610612741,Chicago Bulls,39,27,12,0.692,48.0,41.4,87.0,0.475,...,8,11,21,24,7,28,8,8,10,Chicago Bulls
5,1610612739,Cleveland Cavaliers,42,24,18,0.571,48.0,39.8,85.5,0.465,...,26,19,23,14,1,25,21,4,10,Cleveland Cavaliers
6,1610612742,Dallas Mavericks,41,22,19,0.537,48.2,39.0,86.6,0.450,...,2,26,29,3,19,9,25,11,10,Dallas Mavericks
7,1610612743,Denver Nuggets,39,20,19,0.513,48.1,39.8,86.1,0.462,...,15,20,28,16,13,21,24,15,10,Denver Nuggets
8,1610612765,Detroit Pistons,40,9,31,0.225,48.3,36.6,88.2,0.415,...,28,8,15,21,29,5,29,30,10,Detroit Pistons
9,1610612744,Golden State Warriors,40,30,10,0.750,48.1,40.2,86.0,0.467,...,29,2,11,1,26,29,12,1,10,Golden State Warriors


In [21]:
data, todays_games_uo, frame_ml, home_team_odds, away_team_odds = createTodaysGames(games, df)

Milwaukee Bucks vs Golden State Warriors: 224
Milwaukee Bucks odds: -107
Golden State Warriors odds: -107
Memphis Grizzlies vs Minnesota Timberwolves: 229
Memphis Grizzlies odds: -110
Minnesota Timberwolves odds: -110
New Orleans Pelicans vs LA Clippers: 218
New Orleans Pelicans odds: -110
LA Clippers odds: -110
Brooklyn Nets vs Oklahoma City Thunder: 213.5
Brooklyn Nets odds: -107
Oklahoma City Thunder odds: -107
Denver Nuggets vs Portland Trail Blazers: 216
Denver Nuggets odds: -107
Portland Trail Blazers odds: -107


In [29]:
XGBoost_Runner.xgb_runner(data, todays_games_uo, frame_ml, games, home_team_odds, away_team_odds)

Milwaukee Bucks vs Golden State Warriors (58.2%): OVER 224 (53.0%)
Memphis Grizzlies (77.8%) vs Minnesota Timberwolves: UNDER 229 (55.9%)
New Orleans Pelicans vs LA Clippers (64.0%): UNDER 218 (69.3%)
Brooklyn Nets (88.2%) vs Oklahoma City Thunder: UNDER 213.5 (71.9%)
Denver Nuggets (64.3%) vs Portland Trail Blazers: UNDER 216 (51.6%)
--------------------Expected Value---------------------
Milwaukee Bucks EV: -19.07
Golden State Warriors EV: 12.53
Memphis Grizzlies EV: 48.52
Minnesota Timberwolves EV: -57.61
New Orleans Pelicans EV: -31.24
LA Clippers EV: 22.15
Brooklyn Nets EV: 70.54
Oklahoma City Thunder EV: -77.08
Denver Nuggets EV: 24.4
Portland Trail Blazers EV: -30.94
